In [1]:
import re
import os

import json
import requests

from git2gitee.util import encrypt_pwd

In [5]:
"""
configs 
配置 gitee 用户名，密码。**不要邮箱**
    以及要导入的github 仓库的 url
"""
username = 'mikele'
password = os.getenv('gitee_password')
payload_check_login = {'user_login': username}
repo_url = 'https://github.com/toyourheart163/git2gitee'

In [4]:
raw_headers = """DNT: 1
User-Agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:75.0) Gecko/20100101 Firefox/75.0
Accept-Language: en-US,en;q=0.7,zh-CN;q=0.3
Upgrade-Insecure-Requests: 1
"""

'''utils.py'''
def parse_head(raw_headers):
    headers = {}
    for row in raw_headers.splitlines():
        k, v = row.split(': ')
        headers[k] = v
    return headers

def parse_token(response):
    result = re.search('<meta content="(.*?)" name="csrf-token"', response.text)
    return result.group(1)

In [3]:
s = requests.Session()

base_url = 'https://gitee.com/'
login_url = base_url + 'login'
headers = parse_head(raw_headers)

# get login page, parse token
resp_get_login = s.get(login_url, headers=headers)
token = parse_token(resp_get_login)

# parse public key, generate encrypt password
public_key = json.loads(re.search('gon.encrypt=(.*?);', resp_get_login.text).group(1))['password_key']
encrypt_data = encrypt_pwd(token + '$gitee$' + password, public_key)

In [10]:
# check login account
check_login = False
if check_login:
    url_check_login = base_url + 'check_user_login'
    r_check_login = s.post(url_check_login, data=payload_check_login, headers=headers)
    r_check_login.text

In [11]:
# post login page

login_payload = {
    'encrypt_key': 'password',
    'utf-8': '✓',
    'authenticity_token': token,
    'redirect_to_url': '',
    'user[login]': username,
    'user[remember_me]': '0',
    'encrypt_data[user[password]]': encrypt_data}

resp_login = s.post(login_url, data=login_payload, headers=headers)
print(resp_login.status_code, 'mikele' in resp_login.text)

200 True


In [12]:
url_project = base_url + 'projects/'
url_project_import = url_project + 'import/url'
resp_get_project_import = s.get(url_project_import, headers=headers)
token = parse_token(resp_get_project_import)

check_duplicate = False
check_private = False

check_repo_payload = {'import_url': repo_url}

if check_duplicate:
    url_check_project_duplicate = url_project + 'check_project_duplicate'
    r_check_duplicate = s.get(
        url_check_project_duplicate, params=check_repo_payload, headers=new_headers)
    r_check_duplicate.json()

if check_private:
    url_check_project_private = url_project + 'check_project_private'
    r_check_project_private = s.get(
        url_check_project_private, params=check_repo_payload, headers=new_headers)
    r_check_project_private.json()

In [13]:
def params(repo, username, token, import_url):
    return {
      'utf8': '✓',
      'authenticity_token': token,
      'project[import_url]': import_url,
      'password_sync_code': '',
      'project[name]': repo,
      'project[namespace_path]': username,
      'project[path]': repo,
      'project[description]': '',
      'project[public]': '1'
    }

In [14]:
import_github_project = True

if import_github_project:
    new_headers = headers.copy()
    new_headers['X-CSRF-Token'] = token
    try:
        del new_headers['Upgrade-Insecure-Requests']
        del new_headers['DNT']
    except Exception as e:
        print(e)

    import_project_url = base_url + username + '/projects'
    import_payload = params(
        'git2gitee', 'mikele', token, repo_url)

    r_import = s.post(import_project_url, data=import_payload, headers=new_headers)

In [17]:
if r_import.status_code == requests.codes.ok:
    print('导入成功')

导入成功
